In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("learn.csv")

In [4]:
rational = pd.read_csv("rational_pred2.csv")

In [5]:
del data["Unnamed: 0"]

In [6]:
data.insert_time = pd.to_datetime(data.insert_time)
data.company_enter_month = pd.to_datetime(data.company_enter_month)

In [7]:
del rational["Unnamed: 0"]

In [10]:
c1 = pd.get_dummies(data.income_type)
data = pd.concat([data, c1], axis = 1)

In [11]:
del data['income_type']

In [12]:
data = data[["application_id", "loan_limit", "loan_rate", "credit_score", "company_enter_month","insert_time",
                  "desired_amount", "EARNEDINCOME", "is_applied"]]

In [14]:
test_data = data[data['insert_time']>='2022-05-16']
train_data = data[data['insert_time']<'2022-05-16']

In [15]:
test_data = test_data.groupby('application_id').max().reset_index('inplace' == True, 'drop' == True)
train_data = train_data.groupby('application_id').max().reset_index('inplace' == True, 'drop' == True)

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_3072/3992276412.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.reset_index except for the argument 'level' will be keyword-only.
  test_data = test_data.groupby('application_id').max().reset_index('inplace' == True, 'drop' == True)
/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_3072/3992276412.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.reset_index except for the argument 'level' will be keyword-only.
  train_data = train_data.groupby('application_id').max().reset_index('inplace' == True, 'drop' == True)


In [16]:
test_label = test_data["is_applied"]
del test_data["is_applied"]

In [17]:
train_label = train_data["is_applied"]
del train_data["is_applied"]

In [18]:
from catboost import CatBoostClassifier, Pool

In [20]:
xxx = data.groupby("application_id").max()
k = xxx.is_applied.mean()

In [21]:
model = CatBoostClassifier(iterations=2,
                           depth=5,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)

In [22]:
train = Pool(
    data = train_data,
    label = train_label,
    weight = train_label * (1-2*k) + k
)

In [23]:
model.fit(train)

0:	learn: 0.6663350	total: 78.3ms	remaining: 78.3ms
1:	learn: 0.6602777	total: 95.3ms	remaining: 0us


In [24]:
p = model.predict(test_data)

In [25]:
z = test_data.copy()

In [26]:
z.loc[:, "predict"] = p

In [27]:
appyes = z.loc[z.predict == 1, "application_id"]

In [77]:
zz = data.loc[data.application_id.isin(appyes)]

In [78]:
zz = zz[["application_id", "loan_limit", "loan_rate", "is_applied", "desired_amount"]]

In [79]:
zz = zz.sort_values(by = ["application_id", "loan_rate"])

In [49]:
realdata = data.loc[data.insert_time >= "2022-05-16"]

In [50]:
realdata.loc[:, "predict"] = 0

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_3072/3414315939.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  realdata.loc[:, "predict"] = 0


In [80]:
zz.loc[:, ["sum", "more"]] = 0

In [81]:
zz.loc[:, "predict"] = 0

In [82]:
zz.iloc[0, 6] = zz.iloc[0, 1]

In [83]:
zz

,application_id,loan_limit,loan_rate,is_applied,desired_amount,sum,more,predict
10198527,71,13000000.0,16.6,1.0,5000000.0,0,13000000,0
10052977,92,15000000.0,6.5,0.0,8000000.0,0,0,0
10053008,92,15000000.0,6.5,0.0,8000000.0,0,0,0
10053121,92,15000000.0,6.5,0.0,8000000.0,0,0,0
10052976,92,15000000.0,6.7,0.0,8000000.0,0,0,0
...,...,...,...,...,...,...,...,...
9886970,2167746,3000000.0,19.9,0.0,5000000.0,0,0,0
9886973,2167746,3000000.0,19.9,0.0,5000000.0,0,0,0
8720895,2167790,14000000.0,12.8,0.0,10000000.0,0,0,0
8720858,2167790,42000000.0,15.5,0.0,10000000.0,0,0,0


In [86]:
for i in range (1, len(zz)):
    zz.iloc[i, 5] = zz.iloc[i, 1]
    if (zz.iloc[i, 0] == zz.iloc[i-1, 0]):
        zz.iloc[i, 5] += zz.iloc[i-1, 5]

In [87]:
zz.loc[:,"predict"] = 0

In [88]:
zz["more"] = zz["sum"] > zz.desired_amount

In [89]:
for i in range(2, len(zz)):
    if (zz.iloc[i, 0] == zz.iloc[i-1, 0]):
        if (not zz.iloc[i, 6]):
            zz.iloc[i, 7] = 1
        elif (not zz.iloc[i-1, 6]):
            zz.iloc[i, 7] = 1
        elif (not data.iloc[i-2, 6]):
            data.iloc[i, 7] = 1
    else:
        zz.iloc[i, 7] = 1

In [91]:
zz.predict.sum()

85699

In [92]:
zz.iloc[0, 7] = 1
zz.iloc[1, 7] = 1

In [68]:
from sklearn.metrics import f1_score, confusion_matrix

In [93]:
f1_score(zz.is_applied, zz.predict)

0.4790443922740369

In [95]:
yesindex = zz.loc[zz.predict == 1].index

In [96]:
realdata.loc[yesindex, "predict"] = 1

In [97]:
f1_score(realdata.is_applied, realdata.predict)

0.37308525392298936

In [98]:
confusion_matrix(realdata.is_applied, realdata.predict).ravel()

array([1712122,   49788,   70905,   35913])